Notebook should calculate excited state properties and then score molecules based on performance as excitonic gates

In [ ]:
# Very limited dataset

In [101]:
import os
import sys
import django
import pprint
import pandas as pd
import numpy as np

import collections
import itertools

# setup the django settings file.  Change this to use the settings file that connects you to your desired database
os.environ["DJANGO_SETTINGS_MODULE"] = "djangochem.settings.excitonic_gate_mols"
# this must be run to setup access to the django settings and make database access work etc.
django.setup()

# import the models that you want to access
from pgmols.models import Mol, Calc, Geom
from jobs.models import Job, JobConfig

# this is all setup for the notebook
from IPython.display import HTML
import matplotlib
%matplotlib inline
from rdkit.Chem import AllChem as Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools # headsup: this import change the behavior of dataframes with mols in them
# some global configuration of the pandastools
PandasTools.molRepresentation = 'svg'
PandasTools.molSize = (200,200)

HA_TO_EV = 27.211399
NM_TO_EV=1240


In [102]:
Mol.objects.count()

13

In [103]:
all_tags = set()
for mol in Mol.objects.all():
    tags = mol.tags
    for tag in tags:
        all_tags.add(tag)

In [105]:
print(all_tags)

{'other_mols_candidates', 'best_mols_first_screen'}


In [106]:
Mol.objects.filter(tags__contains=['best_mols_first_screen']).count()

5

In [107]:
# DJANGO QUERY FOR TOTAL NUMBER OF CONFORMERS
Geom.objects.filter(method__name='molecular_mechanics_mmff94').count()

130

In [108]:
def get_sanitized_inchikey(rdkit_mol=None): 
    Chem.SanitizeMol(rdkit_mol)
    inchi_option_string = " -RecMet  -FixedH "
    inchi_string = Chem.MolToInchi(rdkit_mol, options=inchi_option_string)
    inchikey = Chem.InchiToInchiKey(inchi_string)
    return inchikey

In [109]:


# this is a little helper function to render images inside a dataframe
# once again, there are ways to monkey patch the rendering of dataframes, but I am trying to 
# avoid most of that to make things a bit easier to understand

def show(df):
    return HTML(df.to_html(escape=False))

In [110]:
# print all mol tags
#for mol in Mol.objects.all():
#    print(mol.tags)

In [111]:
def make_gate_dict_from_mol(molecule):
    out_dict = {}
    temp_mol_dict = molecule.to_dict()
    out_dict['mol'] = temp_mol_dict['mol']
    out_dict['smiles'] = molecule.smiles
    out_dict['inchikey'] = molecule.inchikey
    oxidized_tags = molecule.tags
    out_dict['tags'] = molecule.tags
    out_dict['pgmol'] = molecule
    conf_count = Geom.objects.filter(mol=molecule,method__name='molecular_mechanics_mmff94').count()  #__ following the reference to method
    out_dict['confcount'] = conf_count
    
    return out_dict

In [112]:
emols_gate_dicts = [make_gate_dict_from_mol(mol) for mol in Mol.objects.filter(tags__contains=['best_mols_first_screen'])]
emols_gate_df = pd.DataFrame(emols_gate_dicts)
#cols = ['mol','smiles','inchikey','tags','confcount']
cols = ['mol','confcount','inchikey']
show(emols_gate_df[cols][0:5])  # show is defined at top of notebook

In [113]:
# Same dictionaries but now keyed by inchi tag, so it can be joined with Calcs more easily
emols_gate_mols_by_inchi = {emols_gate_dict['inchikey']:emols_gate_dict for emols_gate_dict in emols_gate_dicts}

In [114]:
# Need to collect the calculations associated with the molecules that are the first ground state opt
mols = [emols_gate_dict['pgmol'] for emols_gate_dict in emols_gate_dicts]
b3lyp_opt_calcs = Calc.objects.filter(mol__in=mols,method__description='QChem b3lyp/6-31gs DFT').all()

In [115]:
b3lyp_opt_calcs_by_mol = collections.defaultdict(list)
for calc in b3lyp_opt_calcs:
    b3lyp_opt_calcs_by_mol[calc.mol].append(calc)

In [116]:
# Rewrite as a function that returns what you want
def extract_b3lyp_opt_min(mols, calcs_by_mol):
    # Find the minimum for each calc of a given mol
    for mol in mols:
        calcs = calcs_by_mol[mol]
        props = list()
        for calc in calcs:
            prop = calc.props
            parentjob = calc.parentjob
            geoms = Geom.objects.filter(parentjob=parentjob)
            # Line above is inefficient with repeated queries, reorganize to be indexed by parent job
#            print(len(geoms))
            geom = geoms[0]
            prop['geom'] = geom
            props.append(prop)
#            prop['geom'] = geom

        if props:
            min_props = min(props,key=lambda d: d['totalenergy']) #minprops is still a dictionary
            min_energy = min_props['totalenergy']
    
            inchikey = mol.inchikey
        
            key = 'B3LYP 6-31gs Min Energy' 
            emols_gate_mols_by_inchi[inchikey][key] = min_energy
            emols_gate_mols_by_inchi[inchikey]['geom'] = min_props['geom']

In [117]:
extract_b3lyp_opt_min(mols, b3lyp_opt_calcs_by_mol)

In [118]:
emols_gate_df = pd.DataFrame(list(emols_gate_mols_by_inchi.values()))
cols = ['mol','confcount','B3LYP 6-31gs Min Energy']
show(emols_gate_df[cols][0:5])  # show is defined at top of notebook

In [39]:
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

In [40]:
# Now add vertical excitation energies
# Singlets First

In [41]:
b3lyp_vertical_singlet_sp_calcs = Calc.objects.filter(mol__in=mols,method__description='QChem b3lyp/6-31gs TDDFT (TDA) Single Point Vertical').all()




In [42]:
b3lyp_vertical_singlet_sp_calcs_by_geom_id = collections.defaultdict(list)
for calc in b3lyp_vertical_singlet_sp_calcs:
    b3lyp_vertical_singlet_sp_calcs_by_geom_id[calc.parentjob.parentid].append(calc)

In [43]:
vertical_error_count = 0

In [44]:
# Rewrite as a function that returns what you want
def extract_b3lyp_vertical_min_geom(calcs_geom_id):
    # Find the minimum for each calc of a given mol
    
    for mol_dict in emols_gate_mols_by_inchi.values():
        try:
            geom_id = mol_dict['geom'].id
            calc = calcs_geom_id[geom_id][0]
            props = calc.props
            excited_states = props['excitedstates']
            sorted_excited_states = sorted(excited_states, key=lambda k: k['energy']) 
            mol_dict['S1 vertical B3LYP/6-31gs (eV)'] = sorted_excited_states[0]['energy']
            mol_dict['S2 vertical B3LYP/6-31gs (eV)'] = sorted_excited_states[1]['energy']

            mol_dict['S1 to S2 vertical B3LYP/6-31gs (eV)'] = sorted_excited_states[1]['energy'] - sorted_excited_states[0]['energy']
            
            
            mol_dict['S0 S1 vertical oscillator strength'] = sorted_excited_states[0]['interstate_strengths']['0']['strength']
            mol_dict['S0 S1 vertical transition vector'] = sorted_excited_states[0]['interstate_strengths']['0']['vector']

            mol_dict['S1 S2 vertical (S0 geom) oscillator strength'] = sorted_excited_states[0]['interstate_strengths']['2']['strength']
            mol_dict['S1 S2 vertical (S0 geom) transition vector'] = sorted_excited_states[0]['interstate_strengths']['2']['vector']

    
            mol_dict['S0 S2 vertical oscillator strength'] = sorted_excited_states[1]['interstate_strengths']['0']['strength']
            mol_dict['S0 S2 vertical transition vector'] = sorted_excited_states[1]['interstate_strengths']['0']['strength']
    
            vector_1 = mol_dict['S0 S1 vertical transition vector']
            vector_2 = mol_dict['S1 S2 vertical (S0 geom) transition vector']
#            print(vector_1)
 #           print(vector_2)
            mol_dict['S0 S1 vs. S1 S2 vertical transition angle'] = angle_between(vector_1, vector_2)
#            print('Calculated angle!')
#            mol_dict['S0 S1 dipole'] = sorted_excited_states[0]['trans_dipole']
#            mol_dict['S0 S2 dipole'] = sorted_excited_states[1]['trans_dipole']

        except:
            print('Error Occurred')
            pass
        

In [45]:
extract_b3lyp_vertical_min_geom(b3lyp_vertical_singlet_sp_calcs_by_geom_id)

Error Occurred


In [46]:
emols_gate_df = pd.DataFrame(list(emols_gate_mols_by_inchi.values()))
cols = ['mol','S1 to S2 vertical B3LYP/6-31gs (eV)']
show(emols_gate_df[cols][0:5])  # show is defined at top of notebook

In [47]:
# Now add vertical excitation energies
# Vertical Triplets

In [48]:
b3lyp_vertical_triplet_sp_calcs = Calc.objects.filter(mol__in=mols,method__description='QChem b3lyp/6-31gs TDDFT (TDA) Single Point Vertical Triplet').all()
print(len(b3lyp_vertical_triplet_sp_calcs))




75


In [49]:
b3lyp_vertical_triplet_sp_calcs_by_geom_id = collections.defaultdict(list)
for calc in b3lyp_vertical_triplet_sp_calcs:
    b3lyp_vertical_triplet_sp_calcs_by_geom_id[calc.parentjob.parentid].append(calc)
    
    

In [50]:
# Rewrite as a function that returns what you want
def extract_b3lyp_triplet_vertical_min_geom(calcs_geom_id):
    # Find the minimum for each calc of a given mol
    
    for mol_dict in emols_gate_mols_by_inchi.values():
        try:
            geom_id = mol_dict['geom'].id
            calc = calcs_geom_id[geom_id][0]
            props = calc.props
            excited_states = props['excitedstates']
            sorted_excited_states = sorted(excited_states, key=lambda k: k['energy']) 
            mol_dict['T1 vertical B3LYP/6-31gs (eV)'] = sorted_excited_states[0]['energy']
            mol_dict['T2 vertical B3LYP/6-31gs (eV)'] = sorted_excited_states[1]['energy']
            mol_dict['T3 vertical B3LYP/6-31gs (eV)'] = sorted_excited_states[2]['energy']


            mol_dict['T1 T2 vertical (S0 geom) oscillator strength'] = sorted_excited_states[0]['interstate_strengths']['2']['strength']
            mol_dict['T1 T2 vertical (S0 geom) transition vector'] = sorted_excited_states[0]['interstate_strengths']['2']['vector']

            
            mol_dict['T1 T3 vertical (S0 geom) oscillator strength'] = sorted_excited_states[0]['interstate_strengths']['3']['strength']
            mol_dict['T1 T3 vertical (S0 geom) transition moment vector'] = sorted_excited_states[0]['interstate_strengths']['3']['vector']

            mol_dict['T2 T3 vertical (S0 geom) oscillator strength'] = sorted_excited_states[1]['interstate_strengths']['3']['strength']
            mol_dict['T2 T3 vertical (S0 geom) transition vector'] = sorted_excited_states[1]['interstate_strengths']['3']['vector']

    
#            mol_dict['S0 S1 dipole'] = sorted_excited_states[0]['trans_dipole']
#            mol_dict['S0 S2 dipole'] = sorted_excited_states[1]['trans_dipole']

        except:
            print('Error Occurred')
            pass
        

In [51]:
extract_b3lyp_triplet_vertical_min_geom(b3lyp_vertical_triplet_sp_calcs_by_geom_id)

Error Occurred


In [52]:
emols_gate_df = pd.DataFrame(list(emols_gate_mols_by_inchi.values()))
cols = ['mol','confcount','S1 vertical B3LYP/6-31gs (eV)','T1 vertical B3LYP/6-31gs (eV)','S1 to S2 vertical B3LYP/6-31gs (eV)','S2 vertical B3LYP/6-31gs (eV)','T2 vertical B3LYP/6-31gs (eV)']
show(emols_gate_df[cols][0:10])  # show is defined at top of notebook

In [53]:
# Scoring functions 
# 1) dipole[S0->S1] != 0
# Solution, exponentially decaying function as we approach 0. 
# score(x) = (1-exp(-dipole/a)) get a sense for a should be 

In [54]:
import numpy as np

In [55]:
num_mol = len(emols_gate_df.index)
num_mol

5

In [56]:
# Add in S1 opt to get adiabatic excitation energies

In [57]:
# Need to collect the calculations associated with the molecules that are the first ground state opt
mols = [emols_gate_dict['pgmol'] for emols_gate_dict in emols_gate_dicts]
b3lyp_s1_opt_calcs = Calc.objects.filter(mol__in=mols,method__description='QChem b3lyp/6-31gs TDDFT (TDA) s1 opt').all()

In [58]:
b3lyp_s1_opt_calcs_by_mol = collections.defaultdict(list)
for calc in b3lyp_s1_opt_calcs:
    b3lyp_s1_opt_calcs_by_mol[calc.mol].append(calc)

In [59]:
# Rewrite as a function that returns what you want
def extract_b3lyp_s1_opt_min(mols, calcs_by_mol):
    # Find the minimum for each calc of a given mol
    for mol in mols:
        calcs = calcs_by_mol[mol]
        props = list()
        for calc in calcs:
            prop = calc.props
            parentjob = calc.parentjob
            geoms = Geom.objects.filter(parentjob=parentjob)
            # Line above is inefficient with repeated queries, reorganize to be indexed by parent job
#            print(len(geoms))
            geom = geoms[0]
            prop['geom'] = geom
            
#            prop['geom'] = geom
            props.append(prop)
    
    
    
        if props:
            min_props = min(props,key=lambda d: d['totalenergy']) #minprops is still a dictionary
            min_energy = min_props['totalenergy']
    
            inchikey = mol.inchikey
        
            key = 'B3LYP 6-31gs S1 Min Energy' 
            emols_gate_mols_by_inchi[inchikey][key] = min_energy
            emols_gate_mols_by_inchi[inchikey]['s1_min_geom'] = min_props['geom']
            emols_gate_mols_by_inchi[inchikey]['B3LYP 6-31gs S1 Adiabatic Energy (eV)'] = HA_TO_EV*(min_energy-emols_gate_mols_by_inchi[inchikey]['B3LYP 6-31gs Min Energy'])
            # Add in the S1 -> Vertical Transition at the S1 min geom
            excited_states = min_props['excitedstates']
            sorted_excited_states = sorted(excited_states, key=lambda k: k['energy']) 
            s1_s2_vertical_gap = sorted_excited_states[1]['energy'] - sorted_excited_states[0]['energy']
            emols_gate_mols_by_inchi[inchikey]['B3LYP 6-31gs S1 S2 vertical gap (S1 geom) (eV)'] = s1_s2_vertical_gap
            

In [60]:
extract_b3lyp_s1_opt_min(mols, b3lyp_s1_opt_calcs_by_mol)

In [61]:
emols_gate_df = pd.DataFrame(list(emols_gate_mols_by_inchi.values()))

In [62]:
#emols_gate_df['B3LYP 6-31gs S1 Adiabatic Energy (eV)'] = HA_TO_EV*(emols_gate_df['B3LYP 6-31gs S1 Min Energy']-emols_gate_df['B3LYP 6-31gs Min Energy'])



In [63]:
cols = ['mol','S1 vertical B3LYP/6-31gs (eV)','B3LYP 6-31gs S1 Adiabatic Energy (eV)','S2 vertical B3LYP/6-31gs (eV)','S0 S2 vertical oscillator strength','B3LYP 6-31gs S1 S2 vertical gap (S1 geom) (eV)']
show(emols_gate_df[cols][0:5])  # show is defined at top of notebook

In [64]:
# Add in S2 opt to get adiabatic excitation energies
# Need to collect the calculations associated with the molecules that are the second state opt
mols = [emols_gate_dict['pgmol'] for emols_gate_dict in emols_gate_dicts]
b3lyp_s2_opt_calcs = Calc.objects.filter(mol__in=mols,method__description='QChem b3lyp/6-31gs TDDFT (TDA) s2 opt').all()



In [65]:
b3lyp_s2_opt_calcs_by_mol = collections.defaultdict(list)
for calc in b3lyp_s2_opt_calcs:
    b3lyp_s2_opt_calcs_by_mol[calc.mol].append(calc)

In [66]:
# Rewrite as a function that returns what you want
def extract_b3lyp_s2_opt_min(mols, calcs_by_mol):
    # Find the minimum for each calc of a given mol
    for mol in mols:
        calcs = calcs_by_mol[mol]
        props = list()
        for calc in calcs:
            prop = calc.props
            parentjob = calc.parentjob
            geoms = Geom.objects.filter(parentjob=parentjob)
            # Line above is inefficient with repeated queries, reorganize to be indexed by parent job
#            print(len(geoms))
            geom = geoms[0]
            prop['geom'] = geom
            
#            prop['geom'] = geom
            props.append(prop)
    
    
    
        if props:
            min_props = min(props,key=lambda d: d['totalenergy']) #minprops is still a dictionary
            min_energy = min_props['totalenergy']
    
            inchikey = mol.inchikey
        
            key = 'B3LYP 6-31gs S2 Min Energy' 
            emols_gate_mols_by_inchi[inchikey][key] = min_energy
            emols_gate_mols_by_inchi[inchikey]['s2_min_geom'] = min_props['geom']
            emols_gate_mols_by_inchi[inchikey]['B3LYP 6-31gs S2 Adiabatic Energy (eV)'] = HA_TO_EV*(min_energy-emols_gate_mols_by_inchi[inchikey]['B3LYP 6-31gs Min Energy'])
            

In [67]:
extract_b3lyp_s2_opt_min(mols, b3lyp_s2_opt_calcs_by_mol)

In [68]:
emols_gate_df = pd.DataFrame(list(emols_gate_mols_by_inchi.values()))

In [69]:
#emols_gate_df['B3LYP 6-31gs S2 Adiabatic Energy (eV)'] = HA_TO_EV*(emols_gate_df['B3LYP 6-31gs S2 Min Energy']-emols_gate_df['B3LYP 6-31gs Min Energy'])


In [70]:
# Add in T1 opt to get adiabatic excitation energies

In [71]:
# Need to collect the calculations associated with the molecules that are the first ground state opt
mols = [emols_gate_dict['pgmol'] for emols_gate_dict in emols_gate_dicts]
b3lyp_t1_opt_calcs = Calc.objects.filter(mol__in=mols,method__description='QChem b3lyp/6-31gs DFT Triplet Opt').all()

In [72]:
b3lyp_t1_opt_calcs_by_mol = collections.defaultdict(list)
for calc in b3lyp_t1_opt_calcs:
    b3lyp_t1_opt_calcs_by_mol[calc.mol].append(calc)

In [73]:
# Rewrite as a function that returns what you want
def extract_b3lyp_t1_opt_min(mols, calcs_by_mol):
    # Find the minimum for each calc of a given mol
    for mol in mols:
        calcs = calcs_by_mol[mol]
        props = list()
        for calc in calcs:
            prop = calc.props
            parentjob = calc.parentjob
            geoms = Geom.objects.filter(parentjob=parentjob)
            # Line above is inefficient with repeated queries, reorganize to be indexed by parent job
#            print(len(geoms))
            geom = geoms[0]
            prop['geom'] = geom
            
#            prop['geom'] = geom
            props.append(prop)
    
    
        if props:
            min_props = min(props,key=lambda d: d['totalenergy']) #minprops is still a dictionary
            min_energy = min_props['totalenergy']
    
            inchikey = mol.inchikey
        
            key = 'B3LYP 6-31gs T1 Min Energy' 
            emols_gate_mols_by_inchi[inchikey][key] = min_energy
            emols_gate_mols_by_inchi[inchikey]['t1_min_geom'] = min_props['geom']            
            emols_gate_mols_by_inchi[inchikey]['B3LYP 6-31gs T1 Adiabatic Energy (eV)'] = HA_TO_EV*(min_energy-emols_gate_mols_by_inchi[inchikey]['B3LYP 6-31gs Min Energy'])
            
            
            

In [74]:
extract_b3lyp_t1_opt_min(mols, b3lyp_t1_opt_calcs_by_mol)

In [75]:
#emols_gate_df['B3LYP 6-31gs T1 Adiabatic Energy (eV)'] = HA_TO_EV*(emols_gate_df['B3LYP 6-31gs T1 Min Energy']-emols_gate_df['B3LYP 6-31gs Min Energy'])


In [76]:
emols_gate_df = pd.DataFrame(list(emols_gate_mols_by_inchi.values()))

In [77]:
cols = ['mol','S1 vertical B3LYP/6-31gs (eV)','B3LYP 6-31gs S1 Adiabatic Energy (eV)','B3LYP 6-31gs T1 Adiabatic Energy (eV)','S2 vertical B3LYP/6-31gs (eV)','B3LYP 6-31gs S2 Adiabatic Energy (eV)']

#cols = ['mol','S1 vertical B3LYP/6-31gs (eV)','B3LYP 6-31gs S1 Min Energy','B3LYP 6-31gs T1 Min Energy','S2 vertical B3LYP/6-31gs (eV)','B3LYP 6-31gs S2 Adiabatic Energy (eV)']


show(emols_gate_df[cols][0:5])  # show is defined at top of notebook

Plot distributions of properties

In [78]:
emols_gate_df_no_na = emols_gate_df.dropna()

In [79]:
len(emols_gate_df_no_na)

3

In [80]:
emols_gate_df['Log 10 S0 S1 vertical oscillator strength'] = emols_gate_df['S0 S1 vertical oscillator strength'].apply(np.log10)


In [81]:
emols_gate_df['Log 10 S1 S2 vertical (S0 geom) oscillator strength'] = emols_gate_df['S1 S2 vertical (S0 geom) oscillator strength'].apply(np.log10)
emols_gate_df['Log 10 S0 S2 vertical oscillator strength'] = emols_gate_df['S0 S2 vertical oscillator strength'].apply(np.log10)




In [82]:
# Find min and max S1 vertical
print(emols_gate_df['S1 vertical B3LYP/6-31gs (eV)'].max())
print(emols_gate_df['S1 vertical B3LYP/6-31gs (eV)'].min())



2.7052
2.5111


In [83]:
# Add in calibrated ! vertical 
emols_gate_df['Calibrated S1 vertical B3LYP/6-31gs (eV)'] = 0.889762*emols_gate_df['S1 vertical B3LYP/6-31gs (eV)'] + 0.393195


In [84]:
azulene_inchi = get_sanitized_inchikey(Chem.MolFromSmiles('C1=CC2=CC=CC=CC2=C1'))
print(azulene_inchi)

CUFNKYGDVFVPHO-UHFFFAOYNA-N


In [86]:
azulene_s1_vertical = 2.5217

In [87]:

import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np

font_to_use = 'arial'

params = {  'savefig.dpi': 300,
            'text.usetex': False,
            'mathtext.fontset': 'custom',
            'mathtext.cal' : font_to_use,
            'mathtext.rm'  : font_to_use,
            'mathtext.tt'  : font_to_use,
            'mathtext.it'  : font_to_use+':italic',
            'mathtext.bf'  : font_to_use+':bold',
            'mathtext.sf'  : font_to_use,
            'font.family':  font_to_use,
#            'font.weight' : 'bold'
         }



rcParams.update(params)
# rcParams.update({'figure.autolayout': True})

#ax1 = quinone_df['calibrated_voltage'].plot(kind='hist', x='calibrated_voltage', fontsize=14 )
#ax1.set_xlabel("$E^\mathrm{0}$ (DFT)",fontsize=16)
#ax1.set_ylabel("Frequency",fontsize=16)




In [93]:
def plot_figure_with_vertical(dataframe,column,name,vertical_value):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    weights = np.ones_like(dataframe[column].dropna())/float(len(dataframe[column].dropna()))
    ax.hist(dataframe[column].dropna(), weights=weights, edgecolor='black',color='blue')
    ax.axvline(x=vertical_value, color='r', linestyle='dashed', linewidth=2)
    ax.set_xlabel(name,fontsize=16)
    ax.set_ylabel("Normalized Frequency",fontsize=16)
    
    print('Number of entries:'+str(len((dataframe[column].dropna()))))
    fig.canvas.draw()
    plt.show()

In [ ]:
plot_figure_with_vertical(emols_gate_df,'S1 vertical B3LYP/6-31gs (eV)',
                          "B3LYP/6-31G(d) S$_1$ Vertical Excitation Energy (eV)",azulene_s1_vertical)
